In [1]:
!which python 

/home/bachtung/miniconda3/envs/p4ds-env/bin/python


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
point_df = pd.read_csv('diemthi2019_update.csv') 
point_df.head()

,Unnamed: 0,SBD,Ngu Van,Toan Hoc,Ma Ngoai Ngu,Ngoai Ngu,Vat Ly,Hoa Hoc,Sinh Hoc,Lich Su,Dia Ly,GDCD
0,0,24008611,8.00,8.2,N1,7.6,NaN,NaN,NaN,8.25,7.75,8.75
1,1,51000032,6.17,8.6,N1,8.0,8.25,4.5,6.0,NaN,NaN,NaN
2,2,51000005,4.75,5.2,NaN,NaN,NaN,NaN,NaN,4.00,4.25,5.75
3,3,51000021,4.50,4.4,NaN,NaN,NaN,NaN,NaN,2.25,4.75,5.00
4,4,51000013,6.00,5.8,N1,4.4,NaN,NaN,NaN,7.75,8.25,7.50


In [4]:
point_df = point_df[['SBD', 'Toan Hoc', 'Vat Ly', 'Hoa Hoc', 'Sinh Hoc']]
point_df.head()

,SBD,Toan Hoc,Vat Ly,Hoa Hoc,Sinh Hoc
0,24008611,8.2,NaN,NaN,NaN
1,51000032,8.6,8.25,4.5,6.0
2,51000005,5.2,NaN,NaN,NaN
3,51000021,4.4,NaN,NaN,NaN
4,51000013,5.8,NaN,NaN,NaN


In [5]:
num_cols_point = point_df.shape[1]
num_rows_point = point_df.shape[0]

Một dòng là set điểm của một học sinh khóa 2019. Có vẻ không có dòng nào là lạc loài

In [6]:
have_duplicated = point_df.duplicated().any()

- **SBD**: số báo danh của thí sinh.
- **Toán Học**: điểm thi môn toán.
- **Vật Lý**: điểm thi môn Vật Lý.
- **Hóa Học**: điểm thi môn Hóa.
- **Sinh học** điểm thi môn Sinh.

In [7]:
cols_dtype = pd.Series(data=point_df.dtypes, index=point_df.columns)
cols_dtype

SBD           int64
Toan Hoc    float64
Vat Ly      float64
Hoa Hoc     float64
Sinh Hoc    float64
dtype: object

In [8]:
index = ['missing_ratio', 'min', 'max']
nume_columns = ['Toan Hoc', 'Vat Ly', 'Hoa Hoc', 'Sinh Hoc']
nume_col_profile_df = pd.DataFrame(index = index, columns = nume_columns)
nume_col_profile_df = nume_col_profile_df.fillna(0)
nume_col_profile_df.iloc[0,:] = (point_df[nume_columns[:]].isna().sum() / num_rows_point) *100
nume_col_profile_df.iloc[1,:] = point_df[nume_columns[:]].min()
nume_col_profile_df.iloc[2,:] = point_df[nume_columns[:]].max()
nume_col_profile_df

,Toan Hoc,Vat Ly,Hoa Hoc,Sinh Hoc
missing_ratio,0.503664,64.012362,63.596276,64.131472
min,0.000000,0.000000,0.000000,0.000000
max,10.000000,10.000000,10.000000,10.000000


In [20]:
cate_columns = ['SBD']
index = ['missing_ratio', 'num_diff_vals', 'diff_vals']
cate_col_profiles_df = pd.DataFrame(index=index, columns = cate_columns)
cate_col_profiles_df.iloc[0,:] = round((point_df[cate_columns[:]].isna().sum()) / num_rows_point *100, 3)
cate_col_profiles_df.iloc[1,:] = point_df[cate_columns[:]].nunique()
cate_col_profiles_df.iloc[2,:] = cate_col_profiles_df.iloc[2,:].astype(object)
for i in range(len(cate_columns)):
    cate_col_profiles_df.loc['diff_vals',cate_columns[i]] = point_df[cate_columns[i]].unique()


811851

học sinh Việt Nam có xu hướng giỏi Tự Nhiên hơn hay xã hội hơn